Install Libraries

In [1]:
import sys
print(sys.executable)
print(sys.version)
!"{sys.executable}" -m pip install bertopic[visualization] sentence-transformers

c:\Users\A128842\AppData\Local\Programs\Python\Python310\python.exe
3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]


You should consider upgrading via the 'c:\Users\A128842\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import sys
print(sys.executable)
print(sys.version)
!"{sys.executable}" -m pip install bertopic[visualization] -q
!"{sys.executable}" -m pip install matplotlib

c:\Users\A128842\AppData\Local\Programs\Python\Python310\python.exe
3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]


You should consider upgrading via the 'c:\Users\A128842\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\A128842\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [116]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt

Load Data

In [117]:
# Load the cleaned dataset (replace path if needed)
df = pd.read_csv("..\llm_responses_5.csv")
# Let's preview the first few rows
df.head(15)

,Label,Error,Parent Label
0,178_advance_specialist_ed_salary,0,NaN
1,178_advance_specialist_ed_salary,0,NaN
2,-1_leave_fmla_case_status,FMLA paperwork delays,NaN
3,-1_leave_fmla_case_status,FMLA eligibility confusion,NaN
4,-1_leave_fmla_case_status,Leave absence request processing slow,NaN
5,-1_leave_fmla_case_status,Documentation upload issues,NaN
6,-1_leave_fmla_case_status,Payroll discrepancies,NaN
7,-1_leave_fmla_case_status,Leave balance errors,NaN
8,-1_leave_fmla_case_status,FMLA extension denials,NaN
9,-1_leave_fmla_case_status,Benefit enrollment issues,NaN


Drop Rows with 0: those are rows with no useful information

In [118]:
df = df[df["Error"].str.strip() != "0"]
count_zero = (df["Error"].str.strip() == "0").sum()
print(count_zero)

0


Initialize BERTopic with cusom embedding model

In [120]:
# Load a SentenceTransformer model (can swap later for domain-tuned models)
#embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# embedding_model_complex = SentenceTransformer("BAAI/bge-small-en-v1.5")
embedding_model_complex = SentenceTransformer("msmarco-MiniLM-L-6-v3")

Adjust our model knobs to better fit our data

In [122]:
# Import UMAP for dimensionality reduction
from umap import UMAP

# Configure UMAP to preserve more global structure and reduce dimensionality sensibly
umap_model = UMAP(
    n_neighbors=20,    # How many neighbors to consider—higher = more global structure
    n_components=5,    # How many dimensions to reduce to (default is 5)
    min_dist=0.05,      # Minimum distance between points—lower = tighter clusters
    metric="cosine"    # Cosine distance usually works well for embeddings
)

# Import HDBSCAN for clustering
from hdbscan import HDBSCAN

# Configure HDBSCAN to find smaller, denser clusters
hdbscan_model = HDBSCAN(
    min_cluster_size=10,  # Minimum size for a cluster to be formed
    min_samples=2         # Lower = less conservative, more clusters
)

# Finally, create the BERTopic instance with all custom components
topic_model = BERTopic(
    embedding_model=embedding_model_complex,  # SentenceTransformer model
    umap_model=umap_model,            # configured UMAP
    hdbscan_model=hdbscan_model,      # configured HDBSCAN
    verbose=True                      # Print progress messages
)

Fit BERTopic Model

In [152]:
# Fit the model on your text inputs
# topics, probs = topic_model.fit_transform(filtered_df["text_input"].head(1000))
topics, probs = topic_model.fit_transform(df["Error"])

# subset = filtered_df.head(1000).copy()
# subset["Topic"] = topics
# subset["Topic_Probability"] = probs

# # Save for later use if needed
df["Topic"] = topics
df["Topic_Probability"] = probs
print(df.columns)

# OPTIONAL: filter out low-confidence assignments
min_confidence = 0.25
# Filter documents and topics by confidence
filtered_docs = []
filtered_topics = []

for doc, topic, prob in zip(df["Error"], topics, probs):
    if prob >= min_confidence:
        filtered_docs.append(doc)
        filtered_topics.append(topic)

print(filtered_docs)
print(filtered_topics)

2025-07-08 17:41:18,056 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 32/32 [00:02<00:00, 13.16it/s]
2025-07-08 17:41:20,504 - BERTopic - Embedding - Completed ✓
2025-07-08 17:41:20,504 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-08 17:41:22,143 - BERTopic - Dimensionality - Completed ✓
2025-07-08 17:41:22,146 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-08 17:41:22,205 - BERTopic - Cluster - Completed ✓
2025-07-08 17:41:22,213 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-08 17:41:22,273 - BERTopic - Representation - Completed ✓


Index(['Label', 'Error', 'Parent Label', 'Topic', 'Topic_Probability'], dtype='object')
['FMLA paperwork delays', 'FMLA eligibility confusion', 'Leave absence request processing slow', 'Documentation upload issues', 'Payroll discrepancies', 'FMLA extension denials', 'Benefit enrollment issues', 'System maintenance causing delays', 'HRConnect navigation difficulties', 'Leave request form errors', 'FMLA doctor certification issues', 'Leave absence status updates missing', 'FMLA leave date discrepancies', 'FMLA leave exhaustion clarification needed', 'KB', 'FMLA requirements unclear', 'FMLA leave request process complex', 'HR Connect access issues', 'FMLA documentation confusion', 'FMLA leave pay details missing', 'FMLA application reapplication needed', 'EDD FMLA handling frustration', 'FMLA leave schedule adjustment unclear', 'Manager lacks clear reminder system', 'FMLA form instructions unclear', 'FMLA form submission methods not specified', 'SMLA initiation process confusing', 'Retire

Recluster The -1 Set

In [148]:
df_noise = df_with_topics[df_with_topics["Topic"] == -1].copy()
print(f"Number of noise examples: {len(df_noise)}")
print(df_noise["Error"].sample(5).tolist())

Number of noise examples: 139
['Employees miss reporting hours leading to denied requests', 'Union dues deduction visibility unclear', 'Customer left without final decision', 'Leave extension process unclear', 'Timecard code errors frequently uncorrected']


Make a new model - Fit Bertopic

In [138]:
# Import UMAP for dimensionality reduction
from umap import UMAP

# Configure UMAP to preserve more global structure and reduce dimensionality sensibly
umap_model = UMAP(
    n_neighbors=20,    # How many neighbors to consider—higher = more global structure
    n_components=5,    # How many dimensions to reduce to (default is 5)
    min_dist=0,      # Minimum distance between points—lower = tighter clusters
    metric="cosine"    # Cosine distance usually works well for embeddings
)

# Import HDBSCAN for clustering
from hdbscan import HDBSCAN

# Configure HDBSCAN to find smaller, denser clusters
hdbscan_model = HDBSCAN(
    min_cluster_size=3,  # Minimum size for a cluster to be formed
    min_samples=1         # Lower = less conservative, more clusters
)

# Finally, create the BERTopic instance with all custom components
noise_topic_model = BERTopic(
    embedding_model=embedding_model_complex,  # SentenceTransformer model
    umap_model=umap_model,            # configured UMAP
    hdbscan_model=hdbscan_model,      # configured HDBSCAN
    verbose=True                      # Print progress messages
)

# Fit the model on your text inputs
# topics, probs = topic_model.fit_transform(filtered_df["text_input"].head(1000))
noise_topics, noise_probs = noise_topic_model.fit_transform(df_noise["Error"])

2025-07-08 17:25:17,104 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 5/5 [00:00<00:00, 21.67it/s]
2025-07-08 17:25:17,355 - BERTopic - Embedding - Completed ✓
2025-07-08 17:25:17,357 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-08 17:25:17,433 - BERTopic - Dimensionality - Completed ✓
2025-07-08 17:25:17,435 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-08 17:25:17,442 - BERTopic - Cluster - Completed ✓
2025-07-08 17:25:17,446 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-08 17:25:17,456 - BERTopic - Representation - Completed ✓


Review Topics

In [140]:
topic_model_examples = topic_model.get_representative_docs()
noise_topic_model_examples = noise_topic_model.get_representative_docs()

topic_info_reclustered = topic_model.get_topic_info()
label_map_reclustered = dict(zip(
    topic_info_reclustered["Topic"],
    topic_info_reclustered["Name"],
))

topic_info_noise = noise_topic_model.get_topic_info()
label_map_reclustered = dict(zip(
    topic_info_noise["Topic"],
    topic_info_noise["Name"]
))

topic_info_main = topic_model.get_topic_info()
topic_model_examples = topic_model.get_representative_docs()

label_map_main = {}
for topic_id, name in zip(topic_info_main["Topic"], topic_info_main["Name"]):
    label_map_main[topic_id] = {
        "Name": name,
        "Examples": topic_model_examples.get(topic_id, [])
    }

print(label_map_main)

topic_info_noise = noise_topic_model.get_topic_info()
print(topic_info_noise)
noise_topic_model_examples = noise_topic_model.get_representative_docs()

label_map_noise = {}
for topic_id, name in zip(topic_info_noise["Topic"], topic_info_noise["Name"]):
    label_map_noise[topic_id] = {
        "Name": name,
        "Examples": noise_topic_model_examples.get(topic_id, [])
    }

print(label_map_noise)



# labels_main = df_with_topics["My_Final_Label"].unique().tolist()
# labels_noise = df_noise["My_Final_Label"].unique().tolist()
# df_noise["New_Topic"] = noise_topics 
# df_noise["New_Topic_Probability"] = noise_probs
# topic_info = noise_topic_model.get_topic_info()
# print(topic_info)
# rep_docs = noise_topic_model.get_representative_docs()
# print(rep_docs)


{-1: {'Name': '-1_union_escalation_not_for', 'Examples': ['Vague escalation steps for HR access issues', 'Employee unclear on regional HR escalation steps', 'Confusion over plan types and coverage details not resolved effectively']}, 0: {'Name': '0_leave_absence_request_manager', 'Examples': ['Absence request approval process unclear', 'Form submission process lacks clear guidance', 'Leave absence form lacks clear instructions']}, 1: {'Name': '1_fmla_eligibility_leave_criteria', 'Examples': ['FMLA eligibility criteria unclear', 'FMLA eligibility criteria unclear', 'FMLA eligibility unclear']}, 2: {'Name': '2_kb_articles_article_directly', 'Examples': ['KB articles not directly addressing issues', 'KB articles not directly addressing issues', 'KB articles not directly addressing specific issues']}, 3: {'Name': '3_discrepancies_paycheck_discrepancy_pay', 'Examples': ['Pay period deductions vary causing paycheck discrepancies', 'Paycheck discrepancies due to leave date errors', 'Paycheck 

TypeError: unhashable type: 'dict'

Create taxonomy mapping!

In [141]:
taxonomy_mapping = {
    0: "Confusion About Leave Request Submission or Approval",
    1: "Unclear FMLA or Bonding Eligibility Criteria",
    2: "Knowledge Base Articles Missing or Not Relevant",
    3: "Paycheck Errors, Deductions, or Overpayment Disputes",
    4: "Inability to Access HR Systems or Forms",
    5: "Knowledge Base Articles Missing or Not Relevant",
    6: "Employees Unaware of Leave Types or Benefit Interactions",
    7: "Difficulty Providing Documentation or Verifying Identity",
    8: "Lack of Clear Rules for Leave Accrual and Usage",
    9: "Knowledge Base Articles Missing or Not Relevant",
    10: "Complex or Confusing Enrollment Processes",
    11: "Employees Unaware of Leave Types or Benefit Interactions",
    12: "Complex or Confusing Enrollment Processes",
    13: "Delays in Processing or Approving Requests",
    14: "Unclear FMLA or Bonding Eligibility Criteria",
    15: "Confusion About Leave Request Submission or Approval",
    16: "Employees Unaware of Leave Types or Benefit Interactions",
    17: "Complex or Confusing Enrollment Processes",
    18: "Unclear Disability Insurance Procedures",
    19: "Region-Specific Policy or Escalation Confusion",
    20: "Knowledge Base Articles Missing or Not Relevant",
    21: "Inadequate or Vague Communication to Employees",
    22: "Knowledge Base Articles Missing or Not Relevant",
    23: "Generic or Unclassifiable Issues",
    24: "Delays in Processing or Approving Requests",
    25: "Difficulty Providing Documentation or Verifying Identity",
    26: "Unclear FMLA or Bonding Eligibility Criteria",
    27: "Confusion About Leave Request Submission or Approval",
    28: "Paycheck Errors, Deductions, or Overpayment Disputes",
    29: "Paycheck Errors, Deductions, or Overpayment Disputes",
    -1: "Region-Specific Policy or Escalation Confusion",
}

In [155]:
df["Error_Category"] = df["Topic"].map(taxonomy_mapping)
print(df.head(20))
df.to_csv("classified_errors.csv", index=False)

                                  Label  \
2             -1_leave_fmla_case_status   
3             -1_leave_fmla_case_status   
4             -1_leave_fmla_case_status   
5             -1_leave_fmla_case_status   
6             -1_leave_fmla_case_status   
7             -1_leave_fmla_case_status   
8             -1_leave_fmla_case_status   
9             -1_leave_fmla_case_status   
10            -1_leave_fmla_case_status   
11            -1_leave_fmla_case_status   
12            -1_leave_fmla_case_status   
13            -1_leave_fmla_case_status   
14            -1_leave_fmla_case_status   
15            -1_leave_fmla_case_status   
16            -1_leave_fmla_case_status   
17  4_enrollment_dependent_enroll_enrol   
18           17_mother_care_take_family   
19           17_mother_care_take_family   
20           17_mother_care_take_family   
21           17_mother_care_take_family   

                                         Error  Parent Label  Topic  \
2                        

In [102]:
df_noise = pd.DataFrame({
    "Error": noise_texts,
    "New_Topic": noise_topics,
    "New_Topic_Probability": noise_probs
})

In [103]:
print(df_with_topics.head(5))

# print(df_noise.head(5))

df_all = pd.concat([
    df_with_topics[["Error", "Granular_Topic", "Source"]],
    df_noise[["Error", "Granular_Topic", "Source"]]
], ignore_index=True)

                       Label                                  Error  \
2  -1_leave_fmla_case_status                  FMLA paperwork delays   
3  -1_leave_fmla_case_status             FMLA eligibility confusion   
4  -1_leave_fmla_case_status  Leave absence request processing slow   
5  -1_leave_fmla_case_status            Documentation upload issues   
6  -1_leave_fmla_case_status                  Payroll discrepancies   

   Parent Label  Topic  Topic_Probability  
2           NaN     50                1.0  
3           NaN    100                1.0  
4           NaN    115                1.0  
5           NaN     80                1.0  
6           NaN    110                1.0  


KeyError: "['Granular_Topic', 'Source'] not in index"

In [75]:
noise_topic_model.reduce_topics(
    docs=df_noise["Error"].tolist(),
    nr_topics=15
)


# Step 2: Re-transform to get updated assignments
new_topics, new_probs = noise_topic_model.transform(df_noise["Error"].tolist())

# Step 3: Attach
df_noise["Reduced_Topic"] = new_topics
df_noise["Reduced_Topic_Probability"] = new_probs


2025-07-08 15:44:17,299 - BERTopic - Topic reduction - Reducing number of topics
2025-07-08 15:44:17,303 - BERTopic - Topic reduction - Number of topics (15) is equal or higher than the clustered topics(15).
2025-07-08 15:44:17,303 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-08 15:44:17,442 - BERTopic - Representation - Completed ✓
Batches: 100%|██████████| 7/7 [00:00<00:00, 23.19it/s]
2025-07-08 15:44:17,777 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-07-08 15:44:17,777 - BERTopic - Dimensionality - Completed ✓
2025-07-08 15:44:17,777 - BERTopic - Clustering - Approximating new points with `hdbscan_model`


AttributeError: No prediction data was generated

Review Top Topics

In [44]:
import pandas as pd

# Remove row display limits
pd.set_option("display.max_rows", None)

# Get the full topic summary DataFrame
topic_info = topic_model.get_topic_info()

# Print it (no truncation)
print(topic_info)

    Topic  Count                                          Name  \
0      -1    194        -1_process_for_escalation_verification   
1       0    108             0_fmla_eligibility_leave_criteria   
2       1     80                      1_unable_hr_to_hrconnect   
3       2     60  2_discrepancies_paycheck_discrepancy_payment   
4       3     52                  3_leave_form_request_absence   
5       4     38               4_unaware_type_employee_accrual   
6       5     32              5_delays_system_delay_processing   
7       6     30           6_manager_absence_managers_approval   
8       7     30     7_communicated_employees_confuse_policies   
9       8     27        8_upload_document_instructions_unclear   
10      9     23                9_benefit_status_family_denial   
11     10     21            10_causing_confusion_complex_shift   
12     11     19           11_transcripts_provided_articles_kb   
13     12     17              12_retiree_medicare_pension_with   
14     13 

In [47]:
rep_docs = topic_model.get_representative_docs()

In [48]:
import pandas as pd

# Create a list of rows
rows = []
for topic_id, docs in rep_docs.items():
    for doc in docs:
        rows.append({"Topic": topic_id, "Representative Example": doc})

# Convert to DataFrame
df_representative = pd.DataFrame(rows)

# Optional: sort by Topic
df_representative = df_representative.sort_values(by="Topic").reset_index(drop=True)

# Preview
print(df_representative.head(10))


   Topic                             Representative Example
0     -1            Escalation process lacks clear timeline
1     -1  Employee lacks clear guidance on leave applica...
2     -1   Employee unclear on regional HR escalation steps
3      0                  FMLA eligibility criteria unclear
4      0                  FMLA eligibility criteria unclear
5      0                           FMLA eligibility unclear
6      1  Manager unable to access employee timecard in ...
7      1   Employee unable to locate specific leave details
8      1  Employee unable to find specific regional HR c...
9      2  Pay period deductions vary causing paycheck di...


Process the uncategorized ones

In [46]:
# Assuming you have your dataframe
df_noise = df[df["topic"] == -1]
docs_noise = df_noise["Error"].tolist()

KeyError: 'topic'

Populate our Main Dictionary (Filtered_df) with our found topics

In [15]:
filtered_df["topic"] = topics
filtered_df["topic_probability"] = probs

NameError: name 'filtered_df' is not defined

Build Our Dictionary Mapping For Our Database

In [16]:
# Get topic info
topic_info = topic_model.get_topic_info()

# Build mapping {topic_id: topic_name}
topic_to_label = dict(zip(topic_info["Topic"], topic_info["Name"])) 

# print(topic_to_label)
# print(filtered_df["topic"])

# Check the dtype of the topic column
print("Topic column dtype:", filtered_df["topic"].dtype)

# Show example unique topics
print("Example topic values:", filtered_df["topic"].unique()[:10])

# Check one mapping key type
sample_key = list(topic_to_label.keys())[0]
print("Sample key type:", type(sample_key))

unmapped = (
    filtered_df.loc[~filtered_df["topic"].isin(topic_to_label.keys()), "topic"]
    .unique()
)
print("Topic IDs in your data without a mapping:", unmapped)

print(filtered_df["topic_label"].isna().sum())

filtered_df["topic_label"] = filtered_df["topic"].map(topic_to_label)

NameError: name 'filtered_df' is not defined

In [78]:
print(filtered_df.head(20))

                                       Query_Type  Feedback  \
0                                            <NA>  positive   
1                                            <NA>  positive   
2                                            <NA>  positive   
3                                            <NA>  positive   
4                                            <NA>  positive   
5                                            <NA>  positive   
6                                            <NA>  positive   
7           certification validity period inquiry  positive   
8      medical condition and FMLA extension usage  positive   
9                                            <NA>  positive   
10                                           <NA>  positive   
11                                           <NA>  negative   
12                                           <NA>  positive   
13                                           <NA>  positive   
14                                           <NA>  posi

In [68]:
# Subset has the topic assignments for the first 1000 rows
noise_subset = subset[subset["Topic"] == -1]

# Print them
for idx, row in noise_subset.iterrows():
    print(f"{idx}: {row['text_input']}\n")

0:  status salary advance payment ment hour approve expect may 19th bank account

1:  status salary advance payment ment hour approve expect may 19th bank account

2:  status modify payment arrangement document send address wage garnishment issue verify profile document status th document reflect system monday update document

3:  status modify payment arrangement document send address wage garnishment issue verify profile document status th document reflect system monday update document

9:  report thir contribution charles schwab stop deduct paycheck march set contribution percentage profile paycheck history create ticket processing team investigate processing team communicate charles schwab update email week

20:  request * salary advance * employee northern california region step access salary advance request form go work@kp reference employee form salary advance * * employee request salary advance calendar year ensure appear customer employee approver dashboard option * * employee

Visualize the Topics

In [29]:
# Interactive topic overview
topic_model.visualize_topics()

# Keyword bar chart for a given topic number (e.g., topic 3)
topic_model.visualize_barchart(top_n_topics=10)

# Topic similarity hierarchy
topic_model.visualize_hierarchy()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAASFNFjaFjkPxIU0WNoWOQ/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAFMAAAAAAAAAUwAAAAAAAAC7AAAAAAAAALsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAC4HyJZelrXP7gfIll6Wtc/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAOcAAAAAAAAA5wAAAAAAAgEHAAAAAAACAQcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'EhTRY2hY5D/CeqeyLjbsP8J6p7IuNuw/uB8iWXpa1z8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAJMAAAAAAAAAkwAAAAAAAAD7AAAAAAAAAPsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(255,65,54)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAABOuOR4OSTVP0645Hg5JNU/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAS8AAAAAAAIBLwAAAAAAAQFDAAAAAAABAUMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(255,65,54)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAABrkT34teXgP2uRPfi15eA/TrjkeDkk1T8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACARsAAAAAAAIBGwAAAAAAAAE7AAAAAAAAATsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(255,65,54)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'a5E9+LXl4D+nGyQoRpDjP6cbJChGkOM/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABASsAAAAAAAEBKwAAAAAAAwFLAAAAAAADAUsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(255,65,54)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAAELUhMAHvgPwQtSEwAe+A/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAVcAAAAAAAEBVwAAAAAAAwFfAAAAAAADAV8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(255,65,54)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'pxskKEaQ4z/b55teF0fqP9vnm14XR+o/BC1ITAB74D8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAADgT8AAAAAAAOBPwAAAAAAAgFbAAAAAAACAVsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(35,205,205)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAB8t9rwzpbXP3y32vDOltc/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAWsAAAAAAAEBawAAAAAAAwFzAAAAAAADAXMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(35,205,205)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAAAClynsuvWPwAKXKey69Y/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAX8AAAAAAAEBfwAAAAAAA4GDAAAAAAADgYMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
 

Save our DF

In [87]:


filtered_df.to_csv("../data/processed/with_topics.csv", index=False)

Save the Model

In [ ]:
# Save model for later reuse
topic_model.save("../models/bertopic_base_model")

# Save DataFrame with topics
df.to_csv("../data/processed/with_topics.csv", index=False)
